In [ ]:
#import packages
import pandas as pd
import sys
import matplotlib.pyplot as plt
from datetime import datetime
import logging
import numpy as np
import copy

#custom classes
from classes import classes as cl
from functions import functions as fc
from functions import routing as rt
from functions import simulated_annealing as sa
import random

plt.rcParams.update({'font.size': 22})
logging.basicConfig(filename="sa_lvl_worst_mixed.log", level=logging.INFO)

init_path = 'output/initial_solution'
img_exp_path = 'output/graphs/sa_lvl_worst_mixed/'

Initiation

In [ ]:
fc.print_log("Starting initiation")
#create empty dict
initiation_objects = {'dict_depots': '', 'dict_sites': '', 'dict_plants': '', 'dict_jobs': '', 'dict_tours': '', 'list_days': ''}


objects = fc.initiate(initiation_objects,init_path,'final')

#load objects
list_days = objects['list_days']
dict_depots = objects['dict_depots']
dict_sites = objects['dict_sites']
dict_plants = objects['dict_plants']
dict_jobs = objects['dict_jobs']
dict_tours = objects['dict_tours']

#also create lists for convenience
list_depots = [dict_depots[element] for element in dict_depots]
list_sites = [dict_sites[element] for element in dict_sites]
list_plants = [dict_plants[element] for element in dict_plants]
list_jobs = [dict_jobs[element] for element in dict_jobs]
list_tours  = [dict_tours[element] for element in dict_tours]


fc.print_log("Done with initiation")

In the first stage only one depot ist handled, so moves so its all about dropoff and pickup dates

In [ ]:
#ov tours
for depot in dict_depots:
    total_tasks = 0
    total_distance = 0
    for day in list_days:
        total_tasks += dict_tours[depot][day].total_tasks
        total_distance += dict_tours[depot][day].distance
    print("Depot: {}, tasks: {} - distance: {}".format(depot,total_tasks,total_distance))

print("done here")

In [ ]:
print("start here")
fc.print_log("Start Inital Solution Plot")

#only work with one depot
temp_depot = dict_depots['Neuss']
list_depots = [temp_depot]
dict_depots = {'Neuss': temp_depot}



solution_initial = cl.Solution(temp_depot,dict_tours,list_days)

solution_initial.plot_tasks(50,exp_prefix = img_exp_path + 'solution_inital_')
solution_initial.plot_task_proportion(exp_prefix = img_exp_path + 'solution_inital_')
solution_initial.plot_distances(10000,1500,exp_prefix = img_exp_path + 'solution_inital_')

fc.print_log("Done with  Inital Solution Plot")


After having a look at the inital solution, start optimizing

Stage 1: Try to level out the proportions between dropoff and pickup on eatch job

In [ ]:
fc.print_log("Start with Stage 1a")
#dict_tours_temp = copy.deepcopy(dict_tours)
dict_tours_temp = dict_tours
moves_possible = True
move_count = 0
step_count = 0
step_count_curr = 0

#define temperature schedule
temperature = sa.Geometric_Schedule(1,0.95,200)
current_temp = temperature.temp_initial
#define acceptance
avg_tour_length = solution_initial.total_distance / len(list_days)
print(avg_tour_length)
acceptance = sa.NormalizedExponentialAcceptance(avg_tour_length)
#define overall statistic
dict_statistic = {0: [solution_initial.total_distance, temperature.temp_initial,0,0]}



fc.print_log("Start Moves")
while moves_possible and step_count < 10000:
    move_made = False
    for depot in dict_depots:
        for day_org in list_days:

            #see if there are too many pickups in one job
            tour_org = dict_tours_temp[depot][day_org]

            if tour_org.total_pickups > tour_org.total_dropoffs:
                if tour_org.worst_edge_pickup != '':
                    move_job = tour_org.worst_edge_pickup
                    #see if there is space for a pickup, and check if it works with the earliest pickup date of the job
                    for day_new in list_days:
                        tour_new = dict_tours_temp[depot][day_new]
                        if tour_new.total_pickups < tour_new.total_dropoffs\
                                and move_job.end < day_new:

                            #evaluate move
                            distance_delta = sa.evaluate_pickup(tour_org,tour_new,move_job)

                            #check for acceptance - if accepted, reassign
                            if distance_delta<0:

                                #perform move
                                distance_delta = sa.reassign_pickup(tour_org,tour_new,move_job)

                                #adjust move_bol, statistic and leave for loop
                                move_made = True
                                move_count +=1
                                step_count += 1
                                step_count_curr += 1
                                current_temp = temperature.get_temp(step_count_curr)

                                last_total_distance = dict_statistic[step_count-1][0]
                                dict_statistic[step_count] = [last_total_distance + distance_delta,\
                                                              current_temp, move_count, step_count]
                                break
                            else:
                                step_count += 1
                                last_total_distance = dict_statistic[step_count-1][0]
                                dict_statistic[step_count] = [last_total_distance + distance_delta,\
                                                              current_temp, move_count, step_count]
                                move_made = True
                                break


                print("Current: Loop: {} - Moves: {}                                  ".format(step_count,move_count),end='\r')
                if step_count > 10000: break



    moves_possible = move_made

    print("Current: Loop: {} - Moves: {}                                  ".format(step_count,move_count),end='\r')



fc.print_log("Final: Loop: {} - Moves: {}".format(step_count,move_count))

fc.print_log("Done with Moves")
solution_stage1a = cl.Solution(temp_depot,dict_tours_temp,list_days,dev_curr=dict_statistic, dev_tot_old =dict_statistic)

solution_stage1a.plot_tasks(50,exp_prefix = img_exp_path + 'solution_stage1a')
solution_stage1a.plot_task_proportion(exp_prefix = img_exp_path + 'solution_stage1a')
solution_stage1a.plot_developement(exp_prefix = img_exp_path + 'solution_stage1a')


fc.print_log("Done with Stage 1a")
steps_1a = step_count

In [ ]:
print("old distance: %s" % solution_initial.total_distance)
print("new distance: %s" % solution_stage1a.total_distance)

In [ ]:
fc.print_log("Start with Stage 1b")

dict_tours_temp = dict_tours

list_days_reversed = list_days.copy()
list_days_reversed.reverse()


In [ ]:
moves_possible = True
step_count = steps_1a

#define temperature schedule
temperature = sa.Geometric_Schedule(1,0.95,200)
current_temp = temperature.temp_initial
#define acceptance function
avg_tour_length = solution_initial.total_distance / len(list_days)
print(avg_tour_length)
acceptance = sa.NormalizedExponentialAcceptance(avg_tour_length)
#define temp staistic
step_count_curr = 0
dict_statistic_curr = {0: [solution_stage1a.total_distance, temperature.temp_initial]}
moves_possible = True


fc.print_log("Start Moves")
while moves_possible and step_count < 10000:
    moves_possible = False
    for depot in dict_depots:
        for day_org in list_days_reversed:

            #see if there are too many pickups in one job
            tour_org = dict_tours_temp[depot][day_org]

            if tour_org.total_dropoffs > tour_org.total_pickups:
                if tour_org.worst_edge_dropoff:
                    move_job = tour_org.worst_edge_dropoff
                    #see if there is space for a dropoff, and check if it works with the latest dropoff date of the job
                    for day_new in list_days_reversed:
                        tour_new = dict_tours_temp[depot][day_new]
                        if tour_new.total_dropoffs < tour_new.total_pickups\
                                and move_job.start > day_new:

                            moves_possible = True
                            #evaluate move
                            distance_delta = sa.evaluate_dropoff(tour_org,tour_new,move_job)
                            #check for acceptance - if accepted, reassign
                            if distance_delta<0:
                                #perform move
                                distance_delta = sa.reassign_dropoff(tour_org,tour_new,move_job)

                                #adjust move_bol, statistic and leave for loop
                                move_count +=1
                                step_count += 1
                                step_count_curr += 1
                                current_temp = temperature.get_temp(step_count_curr)

                                last_total_distance = dict_statistic[step_count-1][0]
                                dict_statistic[step_count] = [last_total_distance + distance_delta,\
                                                              current_temp, move_count, step_count]

                                break
                            else:
                                step_count += 1
                                last_total_distance = dict_statistic[step_count-1][0]
                                dict_statistic[step_count] = [last_total_distance + distance_delta,\
                                                              current_temp, move_count, step_count]
                                break





                print("Current: Loop: {} - Moves: {}                                  ".format(step_count,move_count),end='\r')
                if step_count > 9999: break



    print("Current: Loop: {} - Moves: {}                                  ".format(step_count,move_count),end='\r')

if step_count > 9999:
    fc.print_log("Loopcount exit")

fc.print_log("Final: Loop: {} - Moves: {}".format(step_count,move_count))

fc.print_log("Done with Moves")
#create new solutions
solution_stage1b = cl.Solution(temp_depot,dict_tours_temp,list_days,dev_curr=dict_statistic)


solution_stage1b.plot_tasks(50,exp_prefix = img_exp_path + 'solution_stage1b')
solution_stage1b.plot_task_proportion(exp_prefix = img_exp_path + 'solution_stage1b')
fc.print_log("Done with Stage 1b")



In [ ]:
print("test")
solution_stage1b = cl.Solution(temp_depot,dict_tours_temp,list_days,dev_curr=dict_statistic, dev_tot_old =dict_statistic)
solution_stage1b.plot_developement(exp_prefix = img_exp_path + 'solution_stage1b')

In [ ]:
print("old distance: %s" % solution_stage1a.total_distance)
print("new distance: %s" % solution_stage1b.total_distance)

In [ ]:
import math

fc.print_log("Start with Stage 2")
dict_tours_temp = dict_tours
moves_possible = True
move_type = 'pickup'
move_count = 0
loop_count = 0
step_count: int = 0
step_count_curr = 0
#check trycount exits for every pair
list_movetypes = ['pair','pickup','dropoff']

#retrieve avg distance
avg_tour_length = solution_initial.total_distance / len(list_days)
print(avg_tour_length)

#define temperature schedule
temperature = sa.Geometric_Schedule(1,0.90,500)
current_temp = temperature.temp_initial
#define acceptance function
acceptance = sa.NormalizedExponentialAcceptance(avg_tour_length)
#define overall statistic
dict_statistic = {0: [solution_initial.total_distance, temperature.temp_initial, 0,0]}

move_type = 'pickup'
bad_move_count = 0
good_move_count = 0
prop_sum = 0

fc.print_log("Start Random Moves with heuristics")
while loop_count < 10000:
    loop_count += 1
    step_count += 1
    move_made = False

    #prefill statistics to avoid key errors
    dict_statistic[step_count] = [dict_statistic[step_count-1][0],current_temp, move_count, step_count]
    current_temp = temperature.get_temp(step_count)
    #stop when all move types are explored to death
    if not list_movetypes:
        #try to restore the list 5 times
        fc.print_log("No more movetypes")
        break

    for depot in dict_depots:

        if move_type == 'pair':
            #find moves
            tour_org, move_job_pickup, move_job_dropoff, pickup_tour_new, dropoff_tour_new = \
                sa.find_pair_move_worst_random(depot,dict_tours_temp,list_days)
        else:
            #find move
            tour_org, move_job, tour_new = sa.find_single_move_worst_random(move_type, depot, dict_tours_temp,list_days)

        #check if trycount exit happend - if yes remove move type
        if not tour_org:
            fc.print_log("Trycount Exit for {}".format(move_type))
            print(list_movetypes)
            list_movetypes.remove(move_type)
            #check if move types are left
            if list_movetypes:
                move_type = random.choice(list_movetypes)
                print("rechose from {}".format(list_movetypes))
                #fill statistics anyway
                dict_statistic[step_count] = [dict_statistic[step_count-1][0],current_temp, move_count, step_count]
                current_temp = temperature.get_temp(step_count)
            break

        if move_type == 'pair':
            #evaluate move
            distance_delta_pickup = sa.evaluate_pickup(tour_org,pickup_tour_new,move_job_pickup)
            distance_delta_dropoff = sa.evaluate_dropoff(tour_org,dropoff_tour_new,move_job_dropoff)
            distance_delta_total = distance_delta_pickup + distance_delta_dropoff
        else:
            #evaluate move
            distance_delta_total = sa.evaluate_move(move_type,tour_org,tour_new,move_job)

        if distance_delta_total < 0:
            #print(\"{} is negative\".format(distance_delta_total))\n",
            good_move_count += 1
        else:
            prop = math.exp(-distance_delta_total / ((avg_tour_length/10) * current_temp))
            bad_move_count +=1
            prop_sum += prop
            prop_avg = prop_sum/bad_move_count
            full_avg = (prop_sum + good_move_count) / (good_move_count + bad_move_count)
            #print("Prob for {} at temp {} is {} - AVG: {} - AVG_full: {} -  Good: {} Bad: {}".format(round(distance_delta_total,2),round(current_temp,2), round(prop,2),round(prop_avg,2),round(full_avg,2), good_move_count,bad_move_count))

        #check for acceptance - if accepted, reassign
        if acceptance.get_acc(current_temp,distance_delta_total):

            if move_type == 'pair':
                #perform moves
                sa.reassign_pickup(tour_org,pickup_tour_new,move_job_pickup)
                sa.reassign_dropoff(tour_org,dropoff_tour_new,move_job_dropoff)
            else:
                #perform move
                sa.reassign_job(move_type,tour_org,tour_new,move_job)

            #adjust move_bol, statistic and leave for loop
            move_made = True
            move_count +=1
            current_temp = temperature.get_temp(step_count)

            last_total_distance = dict_statistic[step_count-1][0]
            dict_statistic[step_count] = [last_total_distance + distance_delta_total,\
                                                               current_temp, move_count,step_count]
        else:
            dict_statistic[step_count] = [dict_statistic[step_count-1][0],current_temp, move_count, step_count]
            current_temp = temperature.get_temp(step_count)
        #change move type
        move_type = random.choice(list_movetypes)


    print("Current: Loop: {} - Moves: {}                                  ".format(loop_count,move_count),end='\r')
    #export statistics every n iterations
    for i in range(20):
        if loop_count == 500 * i + 1:
            fc.print_log("logging at %s" % loop_count)
            solution_stage_1atemp = cl.Solution(temp_depot,dict_tours_temp,list_days,dev_curr=dict_statistic)
            solution_stage_1atemp.plot_tasks(50, exp_prefix = img_exp_path + 'over_time/solution_stage2_L{}_'.format(loop_count),plot=False)
            solution_stage_1atemp.plot_task_proportion(exp_prefix = img_exp_path + 'over_time/solution_stage2_L{}_'.format(loop_count),plot=False)
            solution_stage_1atemp.plot_developement(exp_prefix = img_exp_path + 'over_time/solution_stage2_L{}_'.format(loop_count),plot=False)
            solution_stage_1atemp.plot_distances(10000,1500,exp_prefix = img_exp_path + 'over_time/solution_stage2_L{}_',plot=False)
if loop_count > 9999:
    fc.print_log("Loopcount exit")

fc.print_log("Final: Loop: {} - Moves: {}".format(loop_count,move_count))

fc.print_log("Done with Moves")

solution_stage2 = cl.Solution(temp_depot,dict_tours_temp,list_days,dev_curr=dict_statistic)

solution_stage2.plot_tasks(50,exp_prefix = img_exp_path + 'solution_stage2',plot=False)
solution_stage2.plot_task_proportion(exp_prefix = img_exp_path + 'solution_stage2',plot=False)
solution_stage2.plot_developement(exp_prefix = img_exp_path + 'solution_stage2',plot=False)
solution_stage2.plot_distances(10000,1500,exp_prefix = img_exp_path + 'solution_stage2',plot=False)
fc.dict_to_csv(dict_statistic,img_exp_path, 'sa_random_stage_1a')


fc.print_log("Done with Stage 1a")
steps_1a = step_count

#worst pair heuristic

In [ ]:
solution_stage2 = cl.Solution(temp_depot,dict_tours_temp,list_days,dev_curr=dict_statistic)
fc.dict_to_csv(dict_statistic,img_exp_path, 'solution_stage2')
print("old distance: %s" % solution_initial.total_distance)
print("new distance: %s" % solution_stage1a.total_distance)






In [ ]:
sys.setrecursionlimit(100000)
fc.save_object(solution_stage2, img_exp_path + 'solution_stage1a.pkl')

In [ ]:
print("old distance: %s" % solution_initial.total_distance)
print("new distance: %s" % solution_stage2.total_distance)
